In [1]:
import pandas as pd

In [2]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
# melb_data.head()
melb_df = melb_data.copy()
# melb_df.head()

## Удаление лишних столбцов

In [3]:
melb_df = melb_df.drop(['index', 'Coordinates'], axis = 1)
# melb_df.drop(['index','Coordinates'],axis=1,inplace=True) # Альтернативный вариант
# melb_df.head()

In [4]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
# display(total_rooms)

## Введение нового признака (столбца)

In [5]:
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
# display(melb_df['MeanRoomsSquare'])

In [6]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
# display(melb_df['AreaRatio'])

## Задание 2.4

In [7]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
#
# Cредняя плотность населения страны
#
Population_density = round((1e+6*countries_df['population'] / countries_df['square']).mean(), 2)
Population_density

84.93

## Преобразуем столбец Date в формат datetime

In [8]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# display(melb_df['Date'])

### Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

* date — дата;                                                                    
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;  
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).

In [9]:
years_sold = melb_df['Date'].dt.year
# print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


**Относительная частота продаж для каждого месяца от общего количества продаж**

In [10]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

### Вычисление интервала между двумя временными промежутками

In [11]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
# display(delta_days)

In [12]:
# display(delta_days.dt.days)  # Вывод в формате просто количество дней

In [13]:
# Возраст объекта на момент продажи
#
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
# display(melb_df['AgeBuilding'])

In [14]:
melb_df = melb_df.drop('YearBuilt', axis=1)

### Cколько объектов недвижимости было продано в выходные (суббота и воскресенье)

In [15]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
# melb_df.head()

In [16]:
mask_1 = melb_df['WeekdaySale']== 5
mask_2 = melb_df['WeekdaySale']== 6
# weekend_count = 
melb_df[mask_1 | mask_2]['WeekdaySale'].shape[0]

12822

# ВЫПОЛНЕНИЯ ЗАДАНИЙ 3.4-3.5
* "City" — город, где был замечен НЛО;
* "Colors Reported" — цвет объекта;
* "Shape Reported" — форма объекта;
* "State" — обозначение штата;
* "Time" — время, когда был замечен НЛО (данные отсортированы от старых наблюдений к новым). 

In [17]:
ufo_data = pd.read_csv('data/ufo.csv', sep=',')
ufo_data.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


### В каком году отмечается наибольшее количество случаев наблюдения НЛО в США?

In [18]:
ufo_data['Time'] = pd.to_datetime(ufo_data['Time'], dayfirst=False)
# .dt.year
ufo_data['Time'].dt.year.mode()

0    1999
dtype: int64

### Cредний интервал времени (в днях) между двумя последовательными случаями наблюдения НЛО в штате Невада (NV)

In [19]:
mask = ufo_data['State' ]== 'NV'
timedelta = ufo_data[mask]['Time'].dt.date.diff()
round(timedelta.dt.days.mean())

69

# Создание и преобразование столбцов с помощью функций

In [20]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

In [39]:
street_types = melb_df['Address'].apply(get_street_type)
# display(street_types)

In [40]:
# nlargest(), который возвращает n наибольших значений из Series
#
# melb_df['StreetType'] = street_types.apply(lambda x: x = 'St' if x == 'Strand')
popular_stypes =street_types.value_counts().nlargest(50).index
# print(popular_stypes)

In [41]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
# display(melb_df['StreetType'].head())
# print('Уникальных записей - ', melb_df['StreetType'].nunique())

**Функция get_weekend(weekday), принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.**

In [24]:
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
    
# display(get_weekend(1))    
# display(get_weekend(6)) 
# display(get_weekend(5)) 

In [25]:
sale_weekend = melb_df['WeekdaySale'].apply(get_weekend)
sale_weekend

melb_df['Weekend'] = sale_weekend
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,12,46,5,St,1
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,2,116,3,St,0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,3067,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,3,117,5,St,1
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,2017-03-04,2.5,3067,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,3,47,5,La,1
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,3067,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,6,2,5,St,1


In [26]:
mask_weekend = melb_df['Weekend'] == 1
Price_weekend = melb_df[mask_weekend]['Price'].mean()
round(Price_weekend)

1081199

   **Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.**

   **Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.**

In [27]:
popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
# print(popular_seller)

In [28]:
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')
# display(melb_df['SellerG'])

In [29]:
price_other = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
price_Nelson = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
multyplay_minprice = price_other/price_Nelson
round(1/multyplay_minprice, 1)

1.3

In [30]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
# display(unique_counts)

In [31]:
# display(melb_df.info())

In [32]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
# display(melb_df.info())

In [33]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
# display(melb_df['Type'])

In [34]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [35]:
# display(melb_df.info())

Преобразуйте признак Suburb следующим образом: оставьте в столбце только 119 наиболее популярных пригородов, остальные замените на 'other'.

Приведите данные в столбце Suburb к категориальному типу.

In [36]:
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
# print(popular_suburb)

In [37]:
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')

In [38]:
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
# display(melb_df.info())